In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.graph_objs import Scatter, Line, Layout, Bar

from sklearn.preprocessing import MinMaxScaler

from time import sleep
import datetime

In [ ]:
## 메모리 과부하 걸림 tlqkf
'''## plotly.express 사용, 10시간 단위로 프로세스 관찰
def visualization(x,col,P_value):
    # 프로세스 값들을 0 ~ 1 사이로 스케일링 작업
    scaler = MinMaxScaler()
    x[x.columns[1:-4]] = scaler.fit_transform(x[x.columns[1:-4]])

    #for col in x.filter(like=P_value[-2:] + "_").columns:
    for i in range(len(x)//36000):  # 관찰 시간을 10시간으로 나누어서 시각화
        extract = x[['time', col, P_value]].iloc[36000*i:36000*(i+1)]
        melted = extract.melt(id_vars=['time'], value_vars=[col,P_value], var_name='process', value_name='condition')
        fig = px.line(melted, x="time", y="condition", color="process")
        #fig.update_layout(height=450, width=280*40, title_text="{} operating 81 hours".format(col))
        fig.update_layout(hovermode='x unified', updatemenus=[dict(type = "buttons", direction = "left", buttons=list([dict(args=[{"yaxis.type": "linear"}], label="LINEAR", method="relayout"), dict(args=[{"yaxis.type": "log"}], label="LOG", method="relayout")]))], height=600, width=1200, title_text="{} operating {}/{} hours".format(col,10*(i+1),len(x)//3600))
        fig.show()
        sleep(10)
    return'''

In [ ]:
from TaPR_pkg import etapr

In [ ]:
trainset1 = pd.read_csv('data/HAI 2.0/training/train1.csv')
trainset1

In [ ]:
trainset2 = pd.read_csv('data/HAI 2.0/training/train2.csv')
trainset2

In [ ]:
trainset3 = pd.read_csv('data/HAI 2.0/training/train3.csv')
trainset3

In [ ]:
validset = pd.read_csv('data/HAI 2.0/validation/validation.csv')
validset

In [ ]:
same = (validset.max() == validset.min()).to_frame()
list(same[same[0] == 1].index)

In [ ]:
scaler = MinMaxScaler()
validset[validset.columns[1:-1]] = scaler.fit_transform(validset[validset.columns[1:-1]])

In [ ]:
validset[list(same[same[0] == 1].index)]

In [ ]:
validset[list(same[same[0] == 1].index)]

In [ ]:
corr = validset.corr()
corr.attack.sort_values(ascending=False)

In [ ]:
## matplotlib.pyplot 을 사용해서 시각화
def visualization(x, col, P_value):
    # 프로세스 값들을 0 ~ 1 사이로 스케일링 작업
    scaler = MinMaxScaler()
    x[x.columns[1:-1]] = scaler.fit_transform(x[x.columns[1:-1]])

    font = {'family':'AppleGothic',
            'color':'steelblue',
            'size':20}

    for i in range(len(x)//10800):  # 관찰 시간을 3시간으로 나누어서 시각화
        plt.figure(figsize=(20,12))
        plt.title("{} operating each 3 hours".format(col), fontdict=font)
        plt.ylim(-0.01,1.01)
        plt.plot(x[col].iloc[10800*i:10800*(i+1)], label='{}'.format(col), alpha=.7)
        plt.plot(x[P_value].iloc[10800*i:10800*(i+1)], 'r:', linewidth=2)
        plt.legend()
        plt.ylabel("condition", fontdict=font)
        plt.xlabel("time (sec)", fontdict=font)
        plt.show()

In [ ]:
visualization(validset, 'C01', 'attack')

In [ ]:
from pyculiarity import detect_ts
from pyculiarity.date_utils import date_format
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

def plot_ts(inDF, y,  savepath ):
    fig = plt.figure(figsize = (22,12))

    plt.plot(inDF.index, inDF[y], alpha = 0.4, label = y)
    plt.legend(); plt.xlabel('time'); plt.ylabel(y)
#     plt.savefig(savepath, dpi = 300); plt.close()

def plot_ts_anoms(inDF, savepath):
    fig = plt.figure(figsize = (22,12))
    #plt.ylim(-0.01,1.01)
    plt.plot(inDF.index, inDF['value'], alpha=0.4, label ='value')
    plt.plot(inDF.index, inDF['anoms'],  color='steelblue', alpha=0.4, marker='o', markersize='7',
                markeredgewidth = 1, markerfacecolor='None', markeredgecolor='red',label='anomalies')
    if 'expected_value' in inDF.columns:
        plt.plot(inDF.index, inDF['expected_value'], color = 'c', marker = '^', markersize = '7',
                 markeredgewidth = 1, markerfacecolor='None', markeredgecolor='c',label='expected_value')

    plt.legend(); plt.xlabel('time');plt.ylabel('value')
#     plt.savefig(savepath, dpi = 300); plt.close()

""" Plotting """
# reformat the index and columns
timeS_DF = validset[['C01','attack']]

anomsDF = timeS_DF[timeS_DF.attack == 1]['C01'].to_frame()
anomsDF.columns = ["anoms"]
anomsDF['is_anom'] = True
# anomsDF.columns = ['anom_value','is_anom']

timeS_DF.drop('attack', axis=1, inplace=True)
timeS_DF.columns = ["value"]

merged_DF = pd.merge(left = timeS_DF, right= anomsDF, left_index=True, right_index=True, how = 'left')
# merged_DF.drop('anom_value',axis = 1,inplace=True)

""" Deliverables """
print('>>> the number of anomaly: ', len(anomsDF))

# plot_ts(timeS_DF, 'value', './figures/orig_%s.png' % n_file ) # original data
plot_ts_anoms(merged_DF,'./void') # anoms marked

In [ ]:
## MAD와 generalized ESD로 S-H-ESD 기능을 구현한 부분
for i in range(1, max_outliers + 1):
        if one_tail:
            if upper_tail:
                ares = data.value - data.value.median()
            else:
                ares = data.value.median() - data.value
        else:
            ares = (data.value - data.value.median()).abs()        
        data_sigma = mad(data.value)

        if data_sigma == 0:
            break

        ares = ares/float(data_sigma)
        R = ares.max()
        temp_max_idx = ares[ares == R].index.tolist()[0]
        R_idx[i - 1] = temp_max_idx
        data = data[data.index != R_idx[i - 1]]
        if one_tail:
            p = 1 - alpha / float(n - i + 1)
        else:
            p = 1 - alpha / float(2 * (n - i + 1))
        t = student_t.ppf(p, (n - i - 1))
        lam = t * (n - i) / float(sqrt((n - i - 1 + t**2) * (n - i + 1)))

    if R > lam:
        num_anoms = i